In [1]:
import pandas as pd

In [13]:
train_selected = pd.read_csv('./Stat441_KaggleProject/train_selected_adam.csv')
test_selected = pd.read_csv('./Stat441_KaggleProject/test_selected_adam.csv')
from sklearn.model_selection import train_test_split
train, valid = train_test_split(train_selected, test_size=0.2)
import xgboost as xgb
from sklearn.metrics import mean_squared_error
X_train_xg = train.drop(['label', 'protein'], axis=1)
Y_train_xg = train['label']
X_valid = valid.drop(['label', 'protein'], axis=1)
Y_valid = valid['label']
xg_reg = xgb.XGBClassifier(objective = 'multi:softprob', colsample_bytree = 0.3, learning_rate = 0.8,
                max_depth = 5, alpha = 10, n_estimators = 10, num_class=15)
xg_reg.fit(X_train_xg, Y_train_xg)
valid_preds = xg_reg.predict(X_valid)
test_preds = xg_reg.predict(test_selected.drop('protein', axis=1))

[17:04:31] WARNING: /Users/travis/build/dmlc/xgboost/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'multi:softprob' was changed from 'merror' to 'mlogloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


/Users/adamyuan/miniconda3/lib/python3.7/site-packages/xgboost/sklearn.py:888: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [14]:
(valid_preds == Y_valid).sum() / len(valid_preds)

0.6692015209125475

In [17]:
valid_preds_labeled = pd.DataFrame(valid_preds, columns = ['label'])
valid_preds_labeled['protein'] = valid['protein'].values
valid_preds_grouped = valid_preds_labeled.groupby('protein').agg(lambda x:x.value_counts().index[0])

Y_valid_labeled = pd.DataFrame(Y_valid, columns = ['label'])
Y_valid_labeled['protein'] = valid['protein'].values
Y_valid_grouped = Y_valid_labeled.groupby('protein').agg(lambda x:x.value_counts().index[0])


test_preds_labeled = pd.DataFrame(test_preds, columns = ['label'])
test_preds_labeled['protein'] = test_selected['protein'].values
test_preds_grouped = test_preds_labeled.groupby('protein').agg(lambda x:x.value_counts().index[0])

In [18]:
float((valid_preds_grouped == Y_valid_grouped).sum() / len(valid_preds_grouped))

0.6359649122807017

In [21]:
valid_preds_grouped['label'].value_counts()

0    120
1     79
3     19
6      5
2      5
Name: label, dtype: int64